# 5. LangChain Expression Language(LCEL) 심층 해설


In [3]:
import os
from google.colab import userdata

load_dotenv()

os.environ["OPENAI_API_KEY"]    = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_ENDPOINT"]= os.getenv("LANGCHAIN_ENDPOINT")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

os.environ["OPENAI_API_KEY"]    = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

ModuleNotFoundError: No module named 'google'

In [ ]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 langchain-community==0.3.0 pydantic==2.10.6

INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 

## 5.1. Runnable과 RunnableSequence―LCEL의 가장 기본적인 구성 요소


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
prompt_value = prompt.invoke({"dish": "카레"})
ai_message = model.invoke(prompt_value)
output = output_parser.invoke(ai_message)

print(output)

In [ ]:
chain = prompt | model | output_parser

In [ ]:
output = chain.invoke({"dish": "카레"})
print(output)

### Runnable의 실행 방법―invoke・stream・batch


In [ ]:
chain = prompt | model | output_parser

for chunk in chain.stream({"dish": "카레"}):
    print(chunk, end="", flush=True)

In [ ]:
chain = prompt | model | output_parser

outputs = chain.batch([{"dish": "카레"}, {"dish": "우동"}])
print(outputs)

### LCEL의 "|"로 다양한 Runnable 연결하기


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [ ]:
cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자의 질문에 단계적으로 답변하세요."),
        ("human", "{question}"),
    ]
)

cot_chain = cot_prompt | model | output_parser

In [ ]:
summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "단계적으로 생각한 답변에서 결론만 추출하세요."),
        ("human", "{text}"),
    ]
)

summarize_chain = summarize_prompt | model | output_parser

In [ ]:
cot_summarize_chain = cot_chain | summarize_chain
output = cot_summarize_chain.invoke({"question": "10 + 2 * 3"})
print(output)

## 5.2. RunnableLambda―임의의 함수를 Runnable로 만들기


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnableLambda


def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | RunnableLambda(upper)

ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

### chain 데코레이터를 사용한 RunnableLambda 구현


In [ ]:
from langchain_core.runnables import chain


@chain
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | upper

ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

### RunnableLambda 자동 변환


In [ ]:
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | upper

In [ ]:
ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

### Runnable의 입력 타입과 출력 타입에 주의


In [ ]:
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | upper

# 아래 코드를 실행하면 오류가 발생합니다
output = chain.invoke({"input": "Hello!"})

In [ ]:
chain = prompt | model | StrOutputParser() | upper

In [ ]:
output = chain.invoke({"input": "Hello!"})
print(output)

### (칼럼) 사용자 함수를 stream에 대응시키는 방법


In [ ]:
from typing import Iterator


def upper(input_stream: Iterator[str]) -> Iterator[str]:
    for text in input_stream:
        yield text.upper()


chain = prompt | model | StrOutputParser() | upper

for chunk in chain.stream({"input": "Hello!"}):
    print(chunk, end="", flush=True)

## 5.3. RunnableParallel―여러 Runnable을 병렬로 처리하기


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

In [ ]:
optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 낙관주의자입니다. 사용자의 입력에 대해 낙관적인 의견을 제공하세요."),
        ("human", "{topic}"),
    ]
)
optimistic_chain = optimistic_prompt | model | output_parser

In [ ]:
pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 비관주의자입니다. 사용자의 입력에 대해 비관적인 의견을 제공하세요."),
        ("human", "{topic}"),
    ]
)
pessimistic_chain = pessimistic_prompt | model | output_parser

In [ ]:
import pprint
from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel(
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
)

output = parallel_chain.invoke({"topic": "생성 AI의 진화에 관해"})
pprint.pprint(output)

### RunnableParallel의 출력을 Runnable의 입력으로 연결하기


In [ ]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 객관적 AI입니다. 두 가지 의견을 종합하세요."),
        ("human", "낙관적 의견: {optimistic_opinion}\n비관적 의견: {pessimistic_opinion}"),
    ]
)

In [ ]:
synthesize_chain = (
    RunnableParallel(
        {
            "optimistic_opinion": optimistic_chain,
            "pessimistic_opinion": pessimistic_chain,
        }
    )
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "생성 AI의 진화에 관해"})
print(output)

### RunnableParallel 자동 변환


In [ ]:
synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
    | synthesize_prompt
    | model
    | output_parser
)

In [ ]:
output = synthesize_chain.invoke({"topic": "생성 AI의 진화에 관해"})
print(output)

### RunnableLambda와의 조합―itemgetter를 사용한 예시


In [ ]:
from operator import itemgetter

topic_getter = itemgetter("topic")
topic = topic_getter({"topic": "생성 AI의 진화에 관해"})
print(topic)

In [ ]:
from operator import itemgetter

synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 객관적 AI입니다. {topic}에 대한 두 가지 의견을 종합하세요.",
        ),
        (
            "human",
            "낙관적 의견: {optimistic_opinion}\n비관적 의견: {pessimistic_opinion}",
        ),
    ]
)

synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
        "topic": itemgetter("topic"),
    }
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "생성 AI의 진화에 관해"})
print(output)

## 5.4. RunnablePassthrough―입력을 그대로 출력하기


In [ ]:
import os
from google.colab import userdata

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [ ]:
!pip install tavily-python==0.5.0

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 고려해 질문에 답하세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke("서울의 현재 날씨는?")
print(output)

### assign―RunnableParallel의 출력에 값 추가하기


In [ ]:
import pprint

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | RunnablePassthrough.assign(answer=prompt | model | StrOutputParser())

output = chain.invoke("서울의 현재 날씨는?")
pprint.pprint(output)

In [ ]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

In [ ]:
output = chain.invoke("서울의 현재 날씨는?")
print(output)

#### <보충:pick>


In [ ]:
chain = (
    RunnableParallel(
        {
            "question": RunnablePassthrough(),
            "context": retriever,
        }
    )
    .assign(answer=prompt | model | StrOutputParser())
    .pick(["context", "answer"])
)

In [ ]:
output = chain.invoke("서울의 현재 날씨는?")
print(output)

### (칼럼) astream_events


In [ ]:
# Google Colab에서는 다음 코드의 "async" 부분에 "Use of "async" not allowed outside of async function"이라고 표시되지만, 오류 없이 실행할 수 있습니다

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

async for event in chain.astream_events("서울의 현재 날씨는?", version="v2"):
    print(event, flush=True)

In [ ]:
async for event in chain.astream_events("서울의 현재 날씨는?", version="v2"):
    event_kind = event["event"]

    if event_kind == "on_retriever_end":
        print("=== 검색 결과 ===")
        documents = event["data"]["output"]
        for document in documents:
            print(document)

    elif event_kind == "on_parser_start":
        print("=== 최종 출력 ===")

    elif event_kind == "on_parser_stream":
        chunk = event["data"]["chunk"]
        print(chunk, end="", flush=True)

### (칼럼) Chat history와 Memory


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

chain = prompt | model | StrOutputParser()

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def respond(session_id: str, human_message: str) -> str:
    chat_message_history = SQLChatMessageHistory(
        session_id=session_id, connection="sqlite:///sqlite.db"
    )

    ai_message = chain.invoke(
        {
            "chat_history": chat_message_history.get_messages(),
            "input": human_message,
        }
    )

    chat_message_history.add_user_message(human_message)
    chat_message_history.add_ai_message(ai_message)

    return ai_message

In [ ]:
from uuid import uuid4

session_id = uuid4().hex

output1 = respond(
    session_id=session_id,
    human_message="안녕하세요! 저는 존이라고 합니다!",
)
print(output1)

output2 = respond(
    session_id=session_id,
    human_message="제 이름을 알고 계신가요?",
)
print(output2)